In [1]:
1+1

2

In [2]:
pwd

'/home/ayush/Documents/AI/Projects/Fake-news-detector/research'

In [3]:
cd ..

/home/ayush/Documents/AI/Projects/Fake-news-detector


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_path: str
    local_file_dir: Path
    fake_data: Path
    true_data:Path
    df_output_path:Path


In [5]:
from src.constants import *
from src.utils.common import read_yaml_file,create_dir

In [6]:
class ConfigurationManager:
    def __init__(self,config=CONFIG_FILE_PATH,
                 param=PARAM_FILE_PATH):
        self.config=read_yaml_file(config)
        self.param=read_yaml_file(param)
        create_dir([self.config.root_Artifact])

    def getDataIngestion(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_dir([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_path=config.source_path,
            local_file_dir=Path(config.local_file_dir),
            true_data=Path(config.true_data),
            fake_data=Path(config.fake_data),
            df_output_path = Path(config.df_output_path)
        )
        return data_ingestion_config

In [7]:
import os
import pandas as pd
from src.logging import logger

In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
# api.dataset_download_files('owner/dataset-name', path='data/', unzip=True)

In [9]:
class DataIngestion:
    def __init__(self,config=DataIngestionConfig):
        self.config=config

    def get_data(self):
        """
        Downloads dataset from Kaggle and saves it locally.
        """
        try:
            api.dataset_download_files(
                dataset=self.config.source_path,
                path=self.config.local_file_dir,
                unzip=True
            )
            logger.debug('Data successfully fetched from Kaggle')
        except Exception as e:
            logger.error(f"Failed to download data: {e}")
            raise e
    def concat_data(self):
        """
        Concatenates the fake and true CSV files into one DataFrame, shuffles it, and saves it.
        """
        try:
            # Load CSVs into DataFrames
            true_df = pd.read_csv(self.config.true_data)
            fake_df = pd.read_csv(self.config.fake_data)
            true_df['label']=1
            fake_df['label']=0

            # Concatenate and shuffle
            df = pd.concat([true_df, fake_df], ignore_index=True)
            df = df.sample(frac=1, random_state=42).reset_index(drop=True)

            # Save the final DataFrame
            df.to_csv(self.config.df_output_path, index=False)
            logger.debug('Data successfully concatenated and saved.')
        except Exception as e:
            logger.error(f"Failed to concatenate data: {e}")
            raise e

     

In [10]:
config=ConfigurationManager()
getDataIngestionConfig = config.getDataIngestion()
dataingestion = DataIngestion(config=getDataIngestionConfig)
dataingestion.get_data()
dataingestion.concat_data()

[2025-06-11 20:19:13,768 : INFO : common : Created directory at: artifact]
[2025-06-11 20:19:13,776 : INFO : common : Created directory at: artifact/data_ingestion]
Dataset URL: https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets


In [11]:
pd.read_csv('artifact/data_ingestion/Data.csv')

,title,text,subject,date,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0
...,...,...,...,...,...
44893,Nigeria says U.S. agrees delayed $593 million ...,ABUJA (Reuters) - The United States has formal...,worldnews,"December 27, 2017",1
44894,Boiler Room #62 – Fatal Illusions,Tune in to the Alternate Current Radio Network...,Middle-east,"June 29, 2016",0
44895,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,I m convinced the Freedom From Religion group...,Government News,"Feb 27, 2016",0
44896,Republican tax plan would deal financial hit t...,WASHINGTON (Reuters) - The Republican tax plan...,politicsNews,"November 2, 2017",1
